In [207]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [208]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [209]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from imblearn.over_sampling import SMOTE


# Data Load

In [210]:
#Data Load
train = pd.read_csv('original_data\\train_modified(회).csv').drop(columns=['ID'])
test = pd.read_csv('original_data\\test_modified(회).csv').drop(columns=['ID'])




# Feature Engineering(나이 중요성 강조)

In [211]:
# (1) 나이 구간 생성

mapping = {
    '만18-34세': 26,
    '만35-37세': 36,
    '만38-39세': 38.5,
    '만40-42세': 41,
    '만43-44세': 43.5,
    '만45-50세': 47.5,
    '알 수 없음': np.nan
}

# train, test 데이터프레임에 대해 기존의 '시술 당시 나이' 컬럼은 그대로 두고,
# 매핑 결과를 새로운 컬럼 '시술 당시 나이_numeric'에 저장합니다.
for df in [train, test]:
    df['시술 당시 나이2'] = df['시술 당시 나이'].map(mapping)




# (2) 다항식 항 생성 (제곱, 세제곱)
for df in [train, test]:
    df['age_squared'] = df['시술 당시 나이2'] ** 2
    df['age_cubed'] = df['시술 당시 나이2'] ** 3


# (3) 임계값 기반 이진 변수 (예: 35세 이상을 Advanced Maternal Age로 지정)
advanced_age_categories = ['만35-37세', '만38-39세', '만40-42세', '만43-44세', '만45-50세']

for df in [train, test]:
    df['30대 이상'] = df['시술 당시 나이'].isin(advanced_age_categories).astype(int)

'''# (4) 
for df in [train, test]:
    df["배아 이식 비율"] = df["이식된 배아 수"] / (df["총 생성 배아 수"] + 1)'''
   
'''(4)
mapping = {
    '만18-34세': "young",
    '만35-37세': "young",
    '만38-39세': "middle",
    '만40-42세': "middle",
    '만43-44세': "old",
    '만45-50세': "old",
    '알 수 없음': np.nan
}

# train, test 데이터프레임에 대해 기존의 '시술 당시 나이' 컬럼은 그대로 두고,
# 매핑 결과를 새로운 컬럼 '시술 당시 나이_numeric'에 저장합니다.
for df in [train, test]:
    df['시술 당시 나이3'] = df['시술 당시 나이'].map(mapping)'''



'(4)\nmapping = {\n    \'만18-34세\': "young",\n    \'만35-37세\': "young",\n    \'만38-39세\': "middle",\n    \'만40-42세\': "middle",\n    \'만43-44세\': "old",\n    \'만45-50세\': "old",\n    \'알 수 없음\': np.nan\n}\n\n# train, test 데이터프레임에 대해 기존의 \'시술 당시 나이\' 컬럼은 그대로 두고,\n# 매핑 결과를 새로운 컬럼 \'시술 당시 나이_numeric\'에 저장합니다.\nfor df in [train, test]:\n    df[\'시술 당시 나이3\'] = df[\'시술 당시 나이\'].map(mapping)'

# Feature Engineering(새로운 시술 유형)

In [212]:
train['clean_시술_유형'] = train['시술 유형'].str.replace(r'[^A-Za-z0-9가-힣\s]', '', regex=True)
train['clean_특정_시술_유형'] = train['특정 시술 유형'].str.replace(r'[^A-Za-z0-9가-힣\s]', '', regex=True)

# 3. 두 열을 "_" 구분자로 결합하여 새로운 피처 생성
train['새로운_시술'] = train['clean_시술_유형'] + '_' + train['clean_특정_시술_유형']

train.drop(columns=['clean_시술_유형'], inplace=True)
train.drop(columns=['clean_특정_시술_유형'], inplace=True)


# 4. 결과 확인 (상위 6개 행 출력)
print(train.head())

test['clean_시술_유형'] = test['시술 유형'].str.replace(r'[^A-Za-z0-9가-힣\s]', '', regex=True)
test['clean_특정_시술_유형'] = test['특정 시술 유형'].str.replace(r'[^A-Za-z0-9가-힣\s]', '', regex=True)

# 3. 두 열을 "_" 구분자로 결합하여 새로운 피처 생성
test['새로운_시술'] = test['clean_시술_유형'] + '_' + test['clean_특정_시술_유형']

test.drop(columns=['clean_시술_유형'], inplace=True)
test.drop(columns=['clean_특정_시술_유형'], inplace=True)


# 4. 결과 확인 (상위 6개 행 출력)
print(train.head())

  시술 시기 코드 시술 당시 나이  임신 시도 또는 마지막 임신 경과 연수 시술 유형 특정 시술 유형  배란 자극 여부  \
0   TRZKPL  만18-34세                    NaN   IVF     ICSI         1   
1   TRYBLT  만45-50세                    NaN   IVF     ICSI         0   
2   TRVNRY  만18-34세                    NaN   IVF      IVF         1   
3   TRJXFG  만35-37세                    NaN   IVF     ICSI         1   
4   TRVNRY  만18-34세                    NaN   IVF     ICSI         1   

     배란 유도 유형  단일 배아 이식 여부  착상 전 유전 검사 사용 여부  착상 전 유전 진단 사용 여부  ...  \
0  기록되지 않은 시행          0.0               NaN               0.0  ...   
1      알 수 없음          0.0               NaN               0.0  ...   
2  기록되지 않은 시행          0.0               NaN               0.0  ...   
3  기록되지 않은 시행          0.0               NaN               0.0  ...   
4  기록되지 않은 시행          0.0               NaN               0.0  ...   

   난자 해동 경과일  난자 혼합 경과일  배아 이식 경과일  배아 해동 경과일  임신 성공 여부  시술 당시 나이2  \
0        NaN        0.0        3.0        NaN         0       26.0   
1     

# Bining - 총 배아 생성 수

# 결측치 처리

In [213]:
train['배란 유도 유형'].value_counts()
test['배란 유도 유형'].value_counts()



train = train.drop('배란 유도 유형', axis=1)
test= test.drop('배란 유도 유형', axis=1)

#배란 유도 유형 삭제 

In [214]:
#결측치가 많은 columns 확인후 수동으로 삭제함 
missing_counts = train.isnull().sum().to_frame(name="missing_values")
filtered_columns = missing_counts[missing_counts["missing_values"] >= 190000]
print(filtered_columns)

#train update
train = train.drop(filtered_columns.index, axis=1)
test = test.drop(filtered_columns.index, axis=1)

                       missing_values
임신 시도 또는 마지막 임신 경과 연수          246981
착상 전 유전 검사 사용 여부               253633
PGD 시술 여부                      254172
PGS 시술 여부                      254422
난자 해동 경과일                      254915
배아 해동 경과일                      215982


# Multi one hot code

In [215]:
train['배아 생성 주요 이유'].value_counts()

배아 생성 주요 이유
현재 시술용                 233732
배아 저장용                   9192
기증용, 현재 시술용              3784
난자 저장용                   1959
기증용                      1108
기증용, 배아 저장용               125
배아 저장용, 현재 시술용             83
기증용, 난자 저장용                44
기증용, 배아 저장용, 현재 시술용        20
난자 저장용, 배아 저장용              6
난자 저장용, 현재 시술용              5
연구용, 현재 시술용                 1
난자 저장용, 배아 저장용, 연구용         1
Name: count, dtype: int64

In [216]:
train.isnull().sum()

시술 시기 코드         0
시술 당시 나이         0
시술 유형            0
특정 시술 유형         2
배란 자극 여부         0
              ... 
시술 당시 나이2      329
age_squared    329
age_cubed      329
30대 이상           0
새로운_시술           2
Length: 66, dtype: int64

In [217]:
train['배아 생성 주요 이유']

0         현재 시술용
1         현재 시술용
2         현재 시술용
3         현재 시술용
4         현재 시술용
           ...  
256346    현재 시술용
256347    현재 시술용
256348    현재 시술용
256349    현재 시술용
256350    현재 시술용
Name: 배아 생성 주요 이유, Length: 256351, dtype: object

In [218]:
# 최빈값 계산
most_frequent_value_train = train['배아 생성 주요 이유'].mode()[0]
most_frequent_value_test = test['배아 생성 주요 이유'].mode()[0]  # 테스트셋에서도 따로 구할 수 있음

# 결측치 채우기 (최빈값 사용)
train['배아 생성 주요 이유'].fillna(most_frequent_value_train, inplace=True)
test['배아 생성 주요 이유'].fillna(most_frequent_value_test, inplace=True)

# ✅ 결과 확인
print("결측치 개수 (train):", train['배아 생성 주요 이유'].isna().sum())
print("결측치 개수 (test):", test['배아 생성 주요 이유'].isna().sum())
print("최빈값 (train):", most_frequent_value_train)
print("최빈값 (test):", most_frequent_value_test)


결측치 개수 (train): 0
결측치 개수 (test): 0
최빈값 (train): 현재 시술용
최빈값 (test): 현재 시술용


C:\Users\suest\AppData\Local\Temp\ipykernel_24468\3703942154.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['배아 생성 주요 이유'].fillna(most_frequent_value_test, inplace=True)


In [219]:


# 1. 쉼표 기준으로 나누기 → NaN 값은 빈 리스트로 처리
train['배아 생성 주요 이유'] = train['배아 생성 주요 이유'].fillna("").str.split(', ')
test['배아 생성 주요 이유'] = test['배아 생성 주요 이유'].fillna("").str.split(', ')

# 2. 모든 유니크한 라벨을 찾기 (NaN 방지)
unique_labels = set(label for sublist in train['배아 생성 주요 이유'] if isinstance(sublist, list) for label in sublist)
unique_labels.update(set(label for sublist in test['배아 생성 주요 이유'] if isinstance(sublist, list) for label in sublist))

# 3. 각 라벨별 새로운 컬럼을 생성하고 0/1 값 할당
for label in unique_labels:
    train[label] = train['배아 생성 주요 이유'].apply(lambda x: 1 if isinstance(x, list) and label in x else 0)
    test[label] = test['배아 생성 주요 이유'].apply(lambda x: 1 if isinstance(x, list) and label in x else 0)

# 4. 원본 컬럼 삭제
train.drop(columns=['배아 생성 주요 이유'], inplace=True)
test.drop(columns=['배아 생성 주요 이유'], inplace=True)

# ✅ 결과 확인
print(train.head(5))
print(test.head(5))



  시술 시기 코드 시술 당시 나이 시술 유형 특정 시술 유형  배란 자극 여부  단일 배아 이식 여부  착상 전 유전 진단 사용 여부  \
0   TRZKPL  만18-34세   IVF     ICSI         1          0.0               0.0   
1   TRYBLT  만45-50세   IVF     ICSI         0          0.0               0.0   
2   TRVNRY  만18-34세   IVF      IVF         1          0.0               0.0   
3   TRJXFG  만35-37세   IVF     ICSI         1          0.0               0.0   
4   TRVNRY  만18-34세   IVF     ICSI         1          0.0               0.0   

   남성 주 불임 원인  남성 부 불임 원인  여성 주 불임 원인  ...  시술 당시 나이2  age_squared  \
0           0           0           0  ...       26.0       676.00   
1           0           0           0  ...       47.5      2256.25   
2           0           0           0  ...       26.0       676.00   
3           0           0           0  ...       36.0      1296.00   
4           0           0           0  ...       26.0       676.00   

    age_cubed  30대 이상    새로운_시술  난자 저장용  연구용  배아 저장용  현재 시술용  기증용  
0   17576.000       0  IVF_ICSI     

# X,y 설정

In [220]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

X.head(3)


,시술 시기 코드,시술 당시 나이,시술 유형,특정 시술 유형,배란 자극 여부,단일 배아 이식 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,여성 주 불임 원인,...,시술 당시 나이2,age_squared,age_cubed,30대 이상,새로운_시술,난자 저장용,연구용,배아 저장용,현재 시술용,기증용
0,TRZKPL,만18-34세,IVF,ICSI,1,0.0,0.0,0,0,0,...,26.0,676.00,17576.000,0,IVF_ICSI,0,0,0,1,0
1,TRYBLT,만45-50세,IVF,ICSI,0,0.0,0.0,0,0,0,...,47.5,2256.25,107171.875,1,IVF_ICSI,0,0,0,1,0
2,TRVNRY,만18-34세,IVF,IVF,1,0.0,0.0,0,0,0,...,26.0,676.00,17576.000,0,IVF_IVF,0,0,0,1,0


In [221]:
#Data Pre-processing
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    
    "단일 배아 이식 여부",
    
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    

    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    
    "30대 이상",
    "새로운_시술",
    
    "기증용",  "현재 시술용",  "배아 저장용",  "난자 저장용",  "연구용"
    
    
    
]


In [222]:
#Category -> String
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [223]:
numeric_columns = [
    
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    
    "난자 혼합 경과일",
    "배아 이식 경과일",
    

    "시술 당시 나이2",
    "age_squared",
    "age_cubed",

    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수"
   
    
    
]

X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

# 정규화

In [224]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# 학습 데이터에 fit 후 transform 적용
X_train_encoded[numeric_columns] = scaler.fit_transform(X_train_encoded[numeric_columns])

# 테스트 데이터에는 transform만 적용 (fit X)
X_test_encoded[numeric_columns] = scaler.transform(X_test_encoded[numeric_columns])


In [225]:


# Train-Test Split 
X_train, X_val, y_train, y_val = train_test_split(
    X_train_encoded, y, test_size=0.2, random_state=42, stratify=y
)
#모델에 입력할 수 있도록 pytorch 텐서로 변환
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)

y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1)
# 데이터 로더
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [226]:
import time
from tqdm import tqdm
from sklearn.ensemble import StackingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, roc_auc_score
from joblib import Parallel, delayed

In [227]:
import time
from tqdm import tqdm
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score

In [228]:

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier

from sklearn.ensemble import StackingClassifier

estimators = [

('gb', GradientBoostingClassifier(n_estimators = 10)),

("lgbm", LGBMClassifier(random_state=42)),
("xgb", XGBClassifier(random_state=42)),

("lgbm2", LGBMClassifier(n_estimators=80, random_state=42)),

("ada", AdaBoostClassifier(random_state=42))


]


meta_model = XGBClassifier(n_estimators = 10, random_state = 42) 


stacked_model = StackingClassifier(estimators = estimators,
                                  final_estimator = meta_model,
                                  cv = 3)



# train_x, train_y는 전체 train 데이터를 의미합니다.
stacked_model.fit(X_train, y_train)
stack_pred = stacked_model.predict(X_test_encoded)



[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034717 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 753
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036911 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force

In [229]:
# 확률 예측 (roc_auc_score용)
stack_probs = stacked_model.predict_proba(X_test_encoded)[:, 1]



In [230]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 검증 세트 예측
stack_val_pred = stacked_model.predict(X_val)

# 검증 세트 예측 확률 (양성 클래스에 대한 확률)
stack_val_prob = stacked_model.predict_proba(X_val)[:, 1]

# 정확도 계산
val_accuracy = accuracy_score(y_val, stack_val_pred)
print("Validation Accuracy:", val_accuracy)

# ROC AUC 계산
val_roc_auc = roc_auc_score(y_val, stack_val_prob)
print("Validation ROC AUC:", val_roc_auc)


Validation Accuracy: 0.7445924596750599
Validation ROC AUC: 0.7357489614172037


In [231]:
sample_submission = pd.read_csv('C://Projects//2025_2//LG_aimers//original_data//sample_submission.csv')
sample_submission['probability'] = stack_probs
sample_submission.to_csv('C://Projects//2025_2//LG_aimers//original_data//result_15.999_스태킹_boost.csv', index=False)